In [ ]:
%pylab inline
from utils import *
from ipdb import set_trace
import numpy as np
import torch
import os
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch import autograd
# from torch.autograd import Variable
import nibabel as nib
from torch.utils.data.dataset import Dataset
from torch.utils.data import dataloader
from nilearn import plotting
from ADNI_dataset import *
from BRATS_dataset import *
from ATLAS_dataset import *
from Model_alphaWGAN import *
from Model_VAEGAN import Encoder
from utils import sinkhorn_pointcloud as sp

# Configuration

In [ ]:
BATCH_SIZE=4
TEST_BATCH_SIZE=8
gpu = True
workers = 4

LAMBDA= 10
_eps = 1e-15
Use_BRATS=False
Use_ATLAS = False

#setting latent variable sizes
latent_dim = 1000

gpu_0 = 3
gpu_1 = 3
torch_seed = 4
r_g = torch.manual_seed(torch_seed)
checkpoint_pth = 'wae_checkpoint'

In [ ]:
dataset = ADNIdataset(augmentation=True, img_size=64)
# trainset, testset = data.random_split(dataset, [5, 5], generator=r_g)
train_loader = torch.utils.data.DataLoader(dataset,batch_size=BATCH_SIZE,
                                          shuffle=True,num_workers=workers)
test_loader = torch.utils.data.DataLoader(dataset,batch_size=TEST_BATCH_SIZE,
                                          shuffle=True,num_workers=workers)
# test_loader = torch.utils.data.DataLoader(testset,batch_size=BATCH_SIZE,
#                                           shuffle=True,num_workers=workers)
# if Use_BRATS:
#     #'flair' or 't2' or 't1ce'
#     trainset = BRATSdataset(imgtype='flair')
#     train_loader = torch.utils.data.DataLoader(trainset,batch_size = BATCH_SIZE, shuffle=True,
#                                                num_workers=workers)
# if Use_ATLAS:
#     trainset = ATLASdataset(augmentation=True)
#     train_loader = torch.utils.data.DataLoader(trainset,batch_size=BATCH_SIZE,
#                                           shuffle=True,num_workers=workers)

In [ ]:
G = Generator(noise = latent_dim)
D = Discriminator(is_dis=True)
# E = Encoder(gpu_ind=gpu_0)
E = Discriminator(out_class = latent_dim,is_dis=False, img_size=128)

G.cuda(gpu_0)
D.cuda(gpu_0)
E.cuda(gpu_1)

In [ ]:
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
e_optimizer = optim.Adam(E.parameters(), lr=0.0002)

# Training

In [ ]:
torch.autograd.set_detect_anomaly(True)
#remove Variable usage
# real_y = torch.ones((BATCH_SIZE, 1)).cuda()#async=True))
# fake_y = torch.zeros((BATCH_SIZE, 1)).cuda()#async=True))

# criterion_bce = nn.BCELoss()
criterion_l1 = nn.L1Loss()
criterion_mse = nn.MSELoss()

In [ ]:
# load the highest savepoints of all models
df = load_loss(path=checkpoint_pth)
iteration = load_checkpoint(G, None, E, None, '_iter', path=checkpoint_pth)

In [ ]:
g_iter = 1
d_iter = 3
TOTAL_ITER = 100000
gen_load = inf_train_gen(train_loader)

In [ ]:
while iteration <= TOTAL_ITER:
#     for p in D.parameters():  
#         p.requires_grad = False
    for p in E.parameters():  
        p.requires_grad = True
    for p in G.parameters():  
        p.requires_grad = True
        
    ######i#########################################
    # Train Encoder - Generator 
    ###############################################
    for iters in range(g_iter):
        real_images = gen_load.__next__().cuda(gpu_0)
        _batch_size = real_images.size(0)
        if not _batch_size == BATCH_SIZE: 
            break
        
        z_rand = torch.randn((_batch_size,latent_dim)).cuda(gpu_0)
        z_hat = E(real_images).view(_batch_size,-1)
        x_hat = G(z_hat)
        x_rand = G(z_rand)
        
        ### wasserstein loss between z_e and z_r ###################
        w_dist = 2 * sp.sinkhorn_loss(torch.transpose(z_rand, 0, 1), torch.transpose(z_hat, 0, 1), 0.1, 1000, 100, gpu=gpu_0)
        ###############################################
        ### L2 loss(MSE loss) for reconstruction of the Encoder 
        z_ee, z_re = E(x_hat), E(x_rand)
        z_e_l2, z_r_l2 = 10 * criterion_mse(z_hat, z_ee), 10 * criterion_mse(z_rand, z_re)
        ###############################################
#         d_real_loss = D(x_hat).mean()
#         d_fake_loss = D(x_rand).mean()
#         d_loss = -d_fake_loss-d_real_loss
        l1_loss = 10 * criterion_l1(x_hat.cuda(gpu_1),real_images) #original weight for L1 loss: 10
#         loss1 = l1_loss + d_loss + w_dist + z_e_l2 + z_r_l2  #original weight for c_loss（alpha_gan): 1
        loss1 = l1_loss + z_e_l2 + z_r_l2 + w_dist
#          #original weight for c_loss（alpha_gan): 1
         
        G.zero_grad()
        E.zero_grad()
        if iters<g_iter-1:
            loss1.backward()
        else:
            loss1.backward(retain_graph=True)
        g_optimizer.step()
        g_optimizer.step()
        e_optimizer.step()
        
    ###############################################
    for iters in range(d_iter):
        for p in E.parameters():  
            p.requires_grad = True
        for p in G.parameters():  
            p.requires_grad = False
        loss2 = -2 * E(real_images).mean() + 1 * E(G(E(real_images))).mean() + 1 * E(G(z_rand)).mean()
        E.zero_grad()
        if iters<d_iter-1:
            loss2.backward()
        else:
            loss2.backward(retain_graph=True)
        e_optimizer.step()
    ###############################################
    for p in E.parameters():  
        p.requires_grad = False
    for p in G.parameters():  
        p.requires_grad = True 
    gradient_penalty_r = calc_gradient_penalty(E,real_images, x_rand, cuda_ind=gpu_1) * 1e-2
    gradient_penalty_h = calc_gradient_penalty(E,real_images, x_hat, cuda_ind=gpu_1) * 1e-2
    loss3 = -1 * E(G(E(real_images))).mean() - 1 * E(G(z_rand)).mean() + gradient_penalty_r + gradient_penalty_h
    G.zero_grad()
    loss3.backward()
    g_optimizer.step()
    
    
        
    ###############################################
    # Train D
    ###############################################
#     for p in D.parameters():  
#         p.requires_grad = True
#     for p in E.parameters():  
#         p.requires_grad = False
#     for p in G.parameters():  
#         p.requires_grad = False

#     for iters in range(d_iter):
#         real_images = gen_load.__next__().cuda(gpu_0)
#         _batch_size = real_images.size(0)
        
#         if not _batch_size == BATCH_SIZE: 
#             break
            
#         x_loss2 = -2*D(real_images).mean()+D(x_hat).mean()+D(x_rand).mean()
#         gradient_penalty_r = calc_gradient_penalty(D,real_images, x_rand, cuda_ind=gpu_1)
#         gradient_penalty_h = calc_gradient_penalty(D,real_images, x_hat, cuda_ind=gpu_1)
#         loss2 = x_loss2+gradient_penalty_r+gradient_penalty_h
        
#         d_optimizer.zero_grad()
# #         if iters < d_iter - 1:
# #             loss2.backward()
# #         else:
#         loss2.backward(retain_graph=True)
#         d_optimizer.step()
        
    ###############################################
    # Visualization
    ###############################################

    if iteration % 100 == 0:
#         lossStr = '[{}/{}]'.format(iteration,TOTAL_ITER) + '\n\
#         D: {:<8.3}'.format(loss2.item()) + '\n\
#         En_Ge: {:<8.3}'.format(loss1.item())
        
        lossStr = '[{}/{}]'.format(iteration,TOTAL_ITER) + '\n\
        En_Ge: {:<8.3}'.format(loss1.item())
        
        print('lossStr', lossStr)
        feat = np.squeeze((0.5*real_images[0]+0.5).cpu().numpy())
        feat = nib.Nifti1Image(feat,affine = np.eye(4))
        
        plotting.plot_img(feat,title="X_Real")
        plotting.show()

        feat = np.squeeze((0.5*x_hat[0]+0.5).detach().cpu().numpy())
        feat = nib.Nifti1Image(feat,affine = np.eye(4))
        plotting.plot_img(feat,title="DEC")
        
        feat = np.squeeze((0.5*x_rand[0]+0.5).data.cpu().numpy())
        feat = nib.Nifti1Image(feat,affine = np.eye(4))
        plotting.plot_img(feat,title="x_rand")
        plotting.show()
        
    ###############################################
    # Save the losses
    ###############################################
    loss_dict = {
        'index': [iteration],
        'l1_loss': [l1_loss.item()],
#         'd_recon': [d_real_loss.item()],
#         'd_fake': [d_fake_loss.item()],
#         'd_real': [-2*D(real_images).mean().item()],
        'gp_r': [gradient_penalty_r.item() if gradient_penalty_r else 0],
        'gp_h': [gradient_penalty_h.item() if gradient_penalty_h else 0],
        'loss1': [loss1.item()],
        'loss2': [loss2.item()],
        'loss3': [loss3.item()],
        'z_e_l2': [z_e_l2.item()],
        'z_r_l2': [z_r_l2.item()],
#         'w_dist': [w_dist.item()],
    }
    df = add_loss(df, loss_dict)
        

    ###############################################
    # Model Save
    ###############################################
    if iteration % 1000 == 0 and iteration:
        torch.save(G.state_dict(),f'./{checkpoint_pth}/G_iter'+str(iteration)+'.pth')
#         torch.save(D.state_dict(),f'./{checkpoint_pth}/D_iter'+str(iteration)+'.pth')
        torch.save(E.state_dict(),f'./{checkpoint_pth}/E_iter'+str(iteration)+'.pth')
        write_loss(df, path=checkpoint_pth)
    if iteration % 500 == 0 and iteration:
        viz_pca_tsne(E, dataset, is_tsne=True, latent_size=latent_dim, index=iteration, is_cd=True, batch_size=1, gpu_ind=gpu_1)
        viz_pca_tsne(G, dataset, is_tsne=True, latent_size=latent_dim, index=iteration, gpu_ind=gpu_1, batch_size=4)
        calc_mmd(test_loader, G, iteration, count=1, gpu_ind=gpu_0, mode='rbf', path=checkpoint_pth, no_write=True)
        
        
    iteration += 1
   
    

In [ ]:
write_loss(df, path='test_data')
loss_df = load_loss(path='test_data')
print(loss_df.columns)
loss_df[['loss1', 'loss2', 'loss3']].plot() 
loss_df[['l1_loss']].plot()
loss_df[['z_e_l2','z_r_l2']].plot()
loss_df[['gp_r','gp_h']].plot()
loss_df[['loss1', 'loss2']].plot()
plt.figure(figsize=(400, 400))

In [ ]:
df.index